In [22]:
import pandas as pd
import altair as alt

try:
    df = pd.read_csv('ventas.csv', header=6, encoding='latin-1', sep=';')

except:

    df = pd.read_csv('ventas.csv', header=6, encoding='latin-1', sep=',')

col_super = [c for c in df.columns if '4711' in str(c)][0]
col_ropa = [c for c in df.columns if '4771' in str(c)][0]

df_grafico = df[['Mes y año', col_super, col_ropa]].copy()
df_grafico.columns = ['Fecha_Texto', 'Ventas Supermercado', 'Ventas Ropa']

for col in ['Ventas Supermercado', 'Ventas Ropa']:
    if df_grafico[col].dtype == object:
        df_grafico[col] = df_grafico[col].astype(str).str.replace(',', '.').astype(float)

meses = {'ene': '01', 'feb': '02', 'mar': '03', 'abr': '04', 'may': '05', 'jun': '06',
         'jul': '07', 'ago': '08', 'sep': '09', 'oct': '10', 'nov': '11', 'dic': '12'}

def arreglar_fecha(texto):
    try:
        mes_txt, anio_txt = str(texto).split('-')
        mes_num = meses[mes_txt.lower()]
        return f"20{anio_txt}-{mes_num}-01"
    except:
        return None

df_grafico['Fecha'] = pd.to_datetime(df_grafico['Fecha_Texto'].apply(arreglar_fecha))
df_grafico = df_grafico.dropna(subset=['Fecha'])


df_melted = df_grafico.melt(id_vars=['Fecha'], value_vars=['Ventas Supermercado', 'Ventas Ropa'],
                            var_name='Categoria', value_name='Ventas')

grafico = alt.Chart(df_melted).mark_line(strokeWidth=3).encode(
    x=alt.X('Fecha:T', title='Tiempo'),
    y=alt.Y('Ventas:Q', title='Índice de Ventas Reales (Volumen)'),
    color=alt.Color('Categoria:N',
                    scale=alt.Scale(range=['#000000', '#e18a97']),
                    legend=alt.Legend(title="Gasto")),
    tooltip=['Fecha', 'Categoria', 'Ventas']
).properties(
    title="Compras Reales: ¿Qué consumimos más?",
    width=600, height=400, background="#f9f6ee"
)

grafico.save('grafico_ventas.html')
grafico

alt.Chart(...)